In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, Scale, HORIZONTAL, Toplevel
from PIL import Image, ImageTk, ImageChops, ImageEnhance, ImageDraw, ImageFont, ImageOps
import io
import platform 
import sys
import subprocess
import webbrowser 

In [5]:
# --- CONFIGURARE TEMĂ MODERNĂ ---
COLORS = {
    "bg_main": "#1e1e1e",       # Fundal Canvas
    "bg_panel": "#252526",      # Fundal Sidebar (gri inchis)
    "text_main": "#ffffff",     # Text Alb
    "text_dim": "#aaaaaa",      # Text Gri
    "accent": "#007acc",        # Albastru VS Code
    "accent_hover": "#0098ff",  
    "danger": "#d32f2f",        # Rosu
    "danger_hover": "#f44336",  
    "success": "#2e7d32",       # Verde
    "success_hover": "#4caf50",
    "warning": "#f57c00",       # Portocaliu
    "warning_hover": "#ff9800",
    "purple": "#7b1fa2",        # Mov
    "purple_hover": "#9c27b0",
    "gray_btn": "#3c3c3c",      # Butoane Neutre
    "gray_hover": "#505050"
}

FONT_MAIN = ("Segoe UI", 10)
FONT_BOLD = ("Segoe UI", 10, "bold")

# --- CLASA BUTON ROTUNJIT (FIX BUG DUBLU CLICK) ---
class RoundedButton(tk.Canvas):
    def __init__(self, master, text, command, bg_color, hover_color, width=120, height=35, corner_radius=15, **kwargs):
        # Setam fundalul canvas-ului sa fie acelasi cu al parintelui
        parent_bg = kwargs.pop('bg_parent', COLORS["bg_panel"]) 
        
        super().__init__(master, width=width, height=height, bg=parent_bg, highlightthickness=0, **kwargs)
        
        self.command = command
        self.bg_color = bg_color
        self.hover_color = hover_color
        self.text_str = text
        self.radius = corner_radius
        self.W = width
        self.H = height

        # Desenam forma initiala
        self.rect_id = self.create_rounded_rect(0, 0, self.W, self.H, self.radius, fill=bg_color, outline="")
        
        # Adaugam textul - ATENTIE: state='disabled' lasa click-ul sa treaca prin text direct la buton
        self.text_id = self.create_text(self.W/2, self.H/2, text=self.text_str, fill="white", font=FONT_BOLD, state="disabled")

        # Bindings - DOAR PE CANVAS (eliminam tag_bind care cauza dublura)
        self.bind("<Enter>", self.on_enter)
        self.bind("<Leave>", self.on_leave)
        self.bind("<Button-1>", self.on_click)
        self.bind("<ButtonRelease-1>", self.on_release)

    def create_rounded_rect(self, x1, y1, x2, y2, r, **kwargs):
        points = (x1+r, y1, x1+r, y1, x2-r, y1, x2-r, y1, x2, y1, x2, y1+r, x2, y1+r, x2, y2-r, x2, y2-r, x2, y2, x2-r, y2, x2-r, y2, x1+r, y2, x1+r, y2, x1, y2, x1, y2-r, x1, y2-r, x1, y1+r, x1, y1+r, x1, y1)
        return self.create_polygon(points, smooth=True, **kwargs)

    def on_enter(self, event):
        self.itemconfig(self.rect_id, fill=self.hover_color)
        self.config(cursor="hand2")

    def on_leave(self, event):
        self.itemconfig(self.rect_id, fill=self.bg_color)
        self.config(cursor="")

    def on_click(self, event):
        # Efect vizual de apasare
        self.move(self.text_id, 1, 1) 
        
    def on_release(self, event):
        self.move(self.text_id, -1, -1) 
        if self.command:
            self.command()

# ==========================================
# CLASA LASSO & AI
# ==========================================
class LassoSelectorApp:
    def __init__(self, master_root):
        self.top = Toplevel(master_root)
        self.top.title("Unealtă Selecție (AI & Lasso)")
        self.top.geometry("1000x750")
        self.top.configure(bg=COLORS["bg_main"])
        self.top.grab_set()

        self.source_image = None
        self.tk_source = None
        self.result_image = None
        self.points = []

        # Bara de sus
        btn_frame = tk.Frame(self.top, bg=COLORS["bg_panel"], pady=10)
        btn_frame.pack(fill=tk.X)
        
        # Butoane Stanga
        f_left = tk.Frame(btn_frame, bg=COLORS["bg_panel"])
        f_left.pack(side=tk.LEFT, padx=10)
        RoundedButton(f_left, "1. Încarcă Sursă", self.load_image, COLORS["accent"], COLORS["accent_hover"], width=130).pack(side=tk.LEFT, padx=5)
        RoundedButton(f_left, "Reset", self.reset_selection, COLORS["gray_btn"], COLORS["gray_hover"], width=80).pack(side=tk.LEFT, padx=5)
        
        self.lbl_info = tk.Label(btn_frame, text="(Încarcă imaginea -> Alege metoda)", fg=COLORS["text_dim"], bg=COLORS["bg_panel"], font=FONT_MAIN)
        self.lbl_info.pack(side=tk.LEFT, padx=15)

        # Butoane Dreapta
        f_right = tk.Frame(btn_frame, bg=COLORS["bg_panel"])
        f_right.pack(side=tk.RIGHT, padx=10)
        RoundedButton(f_right, "⚡ Detecție AI", self.auto_remove_background, COLORS["purple"], COLORS["purple_hover"], width=120).pack(side=tk.LEFT, padx=5)
        RoundedButton(f_right, "✂️ Decupare Manuală", self.finish_crop_lasso, COLORS["success"], COLORS["success_hover"], width=150).pack(side=tk.LEFT, padx=5)

        self.canvas = tk.Canvas(self.top, bg="#2d2d2d", cursor="crosshair", highlightthickness=0)
        self.canvas.pack(fill=tk.BOTH, expand=True)

        self.canvas.bind("<ButtonPress-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw_motion)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)
        self.current_line = None
        self.img_offset_x = 0
        self.img_offset_y = 0

    def load_image(self):
        filename = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.jpeg *.png")])
        if filename:
            self.source_image = Image.open(filename).convert("RGBA")
            self.source_image.thumbnail((900, 700))
            self.tk_source = ImageTk.PhotoImage(self.source_image)
            self.canvas.delete("all")
            cx = self.canvas.winfo_width() // 2
            cy = self.canvas.winfo_height() // 2
            self.canvas.create_image(cx, cy, image=self.tk_source, anchor="center", tags="img_src")
            self.img_offset_x = cx - (self.source_image.width // 2)
            self.img_offset_y = cy - (self.source_image.height // 2)
            self.reset_selection()
            self.lbl_info.config(text="Imagine încărcată.")

    def start_draw(self, event):
        self.points = [(event.x, event.y)]
        self.current_line = self.canvas.create_line(event.x, event.y, event.x, event.y, fill="#e74c3c", width=2, tags="lasso_line")

    def draw_motion(self, event):
        if self.points:
            self.points.append((event.x, event.y))
            coords = []
            for p in self.points: coords.extend(p)
            self.canvas.coords(self.current_line, *coords)

    def stop_draw(self, event):
        if len(self.points) > 2:
             self.canvas.create_line(self.points[-1][0], self.points[-1][1], self.points[0][0], self.points[0][1], fill="#e74c3c", width=2, tags="lasso_line")

    def reset_selection(self):
        self.canvas.delete("lasso_line")
        self.points = []

    def finish_crop_lasso(self):
        if not self.source_image or len(self.points) < 3: 
            messagebox.showwarning("Info", "Desenează un contur!")
            return
        mask = Image.new("L", self.source_image.size, 0)
        draw_mask = ImageDraw.Draw(mask)
        real_points = []
        for px, py in self.points:
            real_points.append((px - self.img_offset_x, py - self.img_offset_y))
        draw_mask.polygon(real_points, fill=255, outline=255)
        result = Image.new("RGBA", self.source_image.size, (0,0,0,0))
        result.paste(self.source_image, (0,0), mask)
        bbox = mask.getbbox()
        if bbox: self.result_image = result.crop(bbox)
        else: self.result_image = result
        self.top.destroy()

    def auto_remove_background(self):
        if not self.source_image:
            messagebox.showwarning("Eroare", "Încarcă întâi o imagine!")
            return
        self.lbl_info.config(text="⏳ Inițializare AI...", fg="#f1c40f")
        self.top.update()
        try:
            from rembg import remove
        except ImportError:
            self.lbl_info.config(text="⬇️ Instalare librării AI...", fg="#e74c3c")
            self.top.update()
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "rembg", "onnxruntime"])
                from rembg import remove
            except Exception as e:
                messagebox.showerror("Eroare", str(e))
                return

        self.lbl_info.config(text="⏳ Procesare AI...", fg="#f1c40f")
        self.top.update() 
        try:
            result = remove(self.source_image)
            bbox = result.getbbox()
            if bbox: self.result_image = result.crop(bbox)
            else: self.result_image = result
            self.top.destroy()
        except Exception as e:
            messagebox.showerror("Eroare AI", str(e))
            self.lbl_info.config(text="Eroare.", fg="red")


# ==========================================
# CLASA PRINCIPALA (UI MODERN & ROUNDED)
# ==========================================
class ForensicELAApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Forensic Studio - Expertiză ELA & Analiză Compresie")
        self.root.geometry("1280x900")
        self.root.configure(bg=COLORS["bg_main"]) 

        self.base_image = None
        self.display_image = None
        self.tk_image = None
        self.bg_coords = (0, 0)
        self.overlay_image = None         
        self.original_overlay_image = None 
        self.overlay_id = None
        self.drag_data = {"x": 0, "y": 0}
        self.interaction_mode = None 
        self.text_to_place = ""
        self.current_scale = 1.0
        self.zone_start = None
        self.zone_rect_id = None

        # --- LAYOUT PRINCIPAL ---
        
        # 1. SIDEBAR (STÂNGA)
        sidebar_outer = tk.Frame(root, width=340, bg=COLORS["bg_panel"])
        sidebar_outer.pack(side=tk.LEFT, fill=tk.Y)
        
        self.sb_canvas = tk.Canvas(sidebar_outer, width=320, bg=COLORS["bg_panel"], highlightthickness=0)
        self.sb_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        sb_scroll = tk.Scrollbar(sidebar_outer, orient="vertical", command=self.sb_canvas.yview)
        sb_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.sb_canvas.configure(yscrollcommand=sb_scroll.set)
        
        controls = tk.Frame(self.sb_canvas, bg=COLORS["bg_panel"], padx=15, pady=15)
        self.canvas_window = self.sb_canvas.create_window((0, 0), window=controls, anchor="nw")
        
        controls.bind("<Configure>", lambda e: self.sb_canvas.configure(scrollregion=self.sb_canvas.bbox("all")))
        self.sb_canvas.bind("<Configure>", lambda e: self.sb_canvas.itemconfig(self.canvas_window, width=e.width))
        self._setup_mousewheel(sidebar_outer)

        # 2. ZONA CANVAS (DREAPTA)
        canvas_container = tk.Frame(root, bg="#333") 
        canvas_container.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH)
        
        self.canvas = tk.Canvas(canvas_container, bg="#2d2d2d", cursor="cross", highlightthickness=0)
        self.canvas.pack(expand=True, fill=tk.BOTH, padx=2, pady=2)
        
        self._setup_canvas_bindings()

        # --- POPULARE SIDEBAR (WIDGETS ROTUNJITE) ---

        # TITLU
        tk.Label(controls, text="PANOU CONTROL", font=("Segoe UI", 16, "bold"), bg=COLORS["bg_panel"], fg=COLORS["accent"]).pack(anchor="w", pady=(0, 20))

        # SECTIUNEA 1: GESTIUNE
        self._add_header(controls, "1. GESTIUNE FIȘIERE")
        
        row1 = tk.Frame(controls, bg=COLORS["bg_panel"])
        row1.pack(fill=tk.X, pady=5)
        RoundedButton(row1, "📂 Încarcă", self.load_base_image, COLORS["accent"], COLORS["accent_hover"], width=130).pack(side=tk.LEFT, padx=(0,5))
        RoundedButton(row1, "💾 Salvează", self.save_image, COLORS["accent"], COLORS["accent_hover"], width=130).pack(side=tk.RIGHT, padx=(5,0))
        
        # Buton Reset pe toata latimea (frame wrapper)
        f_res = tk.Frame(controls, bg=COLORS["bg_panel"])
        f_res.pack(fill=tk.X, pady=5)
        RoundedButton(f_res, "🔄 Reset la Original", self.reset_to_original, COLORS["gray_btn"], COLORS["gray_hover"], width=280).pack()
        
        self.lbl_size_comp = tk.Label(controls, text="Mărime fișier: - KB", bg=COLORS["bg_panel"], fg=COLORS["text_main"], font=("Segoe UI", 9))
        self.lbl_size_comp.pack(anchor="w", pady=5)

        self._add_separator(controls)

        # SECTIUNEA 2: FALSIFICARE
        self._add_header(controls, "2. MODIFICARE & FALSIFICARE")
        
        row2 = tk.Frame(controls, bg=COLORS["bg_panel"])
        row2.pack(fill=tk.X, pady=5)
        RoundedButton(row2, "🅰️ Text", self.mode_add_text, COLORS["warning"], COLORS["warning_hover"], width=130).pack(side=tk.LEFT, padx=(0,5))
        RoundedButton(row2, "✂️ Obiect (AI)", self.open_lasso_tool, COLORS["warning"], COLORS["warning_hover"], width=130).pack(side=tk.RIGHT, padx=(5,0))

        tk.Label(controls, text="Rotire Obiect:", bg=COLORS["bg_panel"], fg=COLORS["text_main"], font=("Segoe UI", 9)).pack(anchor="w", pady=(10,0))
        self.rotate_slider = Scale(controls, from_=0, to=360, orient=HORIZONTAL, bg=COLORS["bg_panel"], fg=COLORS["text_main"], 
                                   troughcolor="#444", highlightthickness=0, command=self.on_rotate_slide)
        self.rotate_slider.pack(fill=tk.X)
        self.rotate_slider.config(state=tk.DISABLED)

        self.lbl_status = tk.Label(controls, text="Status: Așteptare...", bg=COLORS["bg_panel"], fg=COLORS["text_dim"], font=("Segoe UI", 9, "italic"), wraplength=260, justify="left")
        self.lbl_status.pack(fill=tk.X, pady=10, ipady=5)

        self._add_separator(controls)

        # SECTIUNEA 3: COMPRESIE
        self._add_header(controls, "3. SIMULARE COMPRESIE")
        
        self.compress_slider = Scale(controls, from_=1, to=100, orient=HORIZONTAL, bg=COLORS["bg_panel"], fg=COLORS["text_main"], 
                                     troughcolor="#444", highlightthickness=0, label="Calitate JPEG (%)")
        self.compress_slider.set(70)
        self.compress_slider.pack(fill=tk.X)

        self.successive_slider = Scale(controls, from_=1, to=50, orient=HORIZONTAL, bg=COLORS["bg_panel"], fg=COLORS["text_main"], 
                                       troughcolor="#444", highlightthickness=0, label="Nr. Re-salvări (Iterații)")
        self.successive_slider.set(1)
        self.successive_slider.pack(fill=tk.X)

        f_comp = tk.Frame(controls, bg=COLORS["bg_panel"])
        f_comp.pack(fill=tk.X, pady=5)
        # Butoane compresie unul sub altul dar centrate
        RoundedButton(f_comp, "Globală (Toată Poza)", self.apply_compression_global, COLORS["danger"], COLORS["danger_hover"], width=280).pack(pady=3)
        RoundedButton(f_comp, "Succesivă (Loop)", self.apply_successive_compression, "#b71c1c", "#d32f2f", width=280).pack(pady=3)
        RoundedButton(f_comp, "Doar Zonă (Selecție)", self.mode_local_compression, "#e53935", "#ef5350", width=280).pack(pady=3)

        self._add_separator(controls)

        # SECTIUNEA 4: ELA
        self._add_header(controls, "4. ANALIZĂ ELA (EXPERTIZĂ)")
        
        self.ela_scale = Scale(controls, from_=10, to=100, orient=HORIZONTAL, bg=COLORS["bg_panel"], fg=COLORS["text_main"], 
                               troughcolor="#444", highlightthickness=0, label="Amplificare (Glow)")
        self.ela_scale.set(40)
        self.ela_scale.pack(fill=tk.X)
        
        self.ela_quality = Scale(controls, from_=50, to=100, orient=HORIZONTAL, bg=COLORS["bg_panel"], fg=COLORS["text_main"], 
                                 troughcolor="#444", highlightthickness=0, label="Calitate Referință")
        self.ela_quality.set(95)
        self.ela_quality.pack(fill=tk.X)

        f_ela = tk.Frame(controls, bg=COLORS["bg_panel"])
        f_ela.pack(fill=tk.X, pady=10)
        RoundedButton(f_ela, "🔍 RULEAZĂ ELA (LOCAL)", self.run_ela, COLORS["purple"], COLORS["purple_hover"], width=280).pack()
        
        # --- BUTOANE LINK WEB ---
        tk.Label(controls, text="Validare Externă:", bg=COLORS["bg_panel"], fg=COLORS["text_dim"], font=("Segoe UI", 9)).pack(anchor="w", pady=(5,0))
        
        f_web = tk.Frame(controls, bg=COLORS["bg_panel"])
        f_web.pack(fill=tk.X, pady=2)
        RoundedButton(f_web, "🌐 FotoForensics.com", self.open_fotoforensics, COLORS["gray_btn"], COLORS["gray_hover"], width=280).pack(pady=2)
        RoundedButton(f_web, "🌐 Forensically (29a.ch)", self.open_forensically, COLORS["gray_btn"], COLORS["gray_hover"], width=280).pack(pady=2)
        
        tk.Label(controls, bg=COLORS["bg_panel"], height=3).pack()

    # --- UI HELPERS ---
    def _add_header(self, parent, text):
        tk.Label(parent, text=text, font=("Segoe UI", 10, "bold"), bg=COLORS["bg_panel"], fg=COLORS["text_dim"]).pack(anchor="w", pady=(15, 5))
    
    def _add_separator(self, parent):
        tk.Frame(parent, height=1, bg="#444").pack(fill=tk.X, pady=10)

    def _setup_mousewheel(self, widget):
        widget.bind('<Enter>', lambda e: self._bind_wheel())
        widget.bind('<Leave>', lambda e: self._unbind_wheel())

    def _bind_wheel(self):
        self.sb_canvas.bind_all("<MouseWheel>", self._on_mousewheel)
        if platform.system() == "Linux":
            self.sb_canvas.bind_all("<Button-4>", self._on_mousewheel)
            self.sb_canvas.bind_all("<Button-5>", self._on_mousewheel)
    
    def _unbind_wheel(self):
        self.sb_canvas.unbind_all("<MouseWheel>")
        if platform.system() == "Linux":
            self.sb_canvas.unbind_all("<Button-4>")
            self.sb_canvas.unbind_all("<Button-5>")

    def _on_mousewheel(self, event):
        if platform.system() == "Windows":
             self.sb_canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        else:
            if event.num == 4: self.sb_canvas.yview_scroll(-1, "units")
            elif event.num == 5: self.sb_canvas.yview_scroll(1, "units")

    def _setup_canvas_bindings(self):
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.canvas.bind("<B1-Motion>", self.on_canvas_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_canvas_release)
        self.canvas.bind("<Button-3>", self.on_right_click)
        if platform.system() == "Linux":
            self.canvas.bind("<Button-4>", self.on_mouse_wheel_zoom)
            self.canvas.bind("<Button-5>", self.on_mouse_wheel_zoom)
        else:
            self.canvas.bind("<MouseWheel>", self.on_mouse_wheel_zoom)

    # --- FUNCTIONALITY ---

    def update_size_label(self):
        if self.display_image:
            buf = io.BytesIO()
            self.display_image.save(buf, "JPEG", quality=95)
            s = buf.tell() / 1024.0
            self.lbl_size_comp.config(text=f"Mărime fișier: {s:.1f} KB")

    def apply_transformations(self):
        if self.original_overlay_image and self.overlay_id:
            img = self.original_overlay_image.copy()
            angle = self.rotate_slider.get()
            if angle != 0: img = img.rotate(angle, expand=True, resample=Image.BICUBIC)
            if self.current_scale != 1.0:
                new_w = int(img.width * self.current_scale)
                new_h = int(img.height * self.current_scale)
                img = img.resize((max(1, new_w), max(1, new_h)), Image.LANCZOS)
            self.overlay_image = img
            coords = self.canvas.coords(self.overlay_id)
            cx, cy = coords[0], coords[1]
            self.tk_overlay = ImageTk.PhotoImage(self.overlay_image)
            self.canvas.itemconfig(self.overlay_id, image=self.tk_overlay)
            self.canvas.coords(self.overlay_id, cx, cy)

    def on_rotate_slide(self, val):
        if self.interaction_mode == "move_object": self.apply_transformations()

    def on_mouse_wheel_zoom(self, event):
        if self.interaction_mode == "move_object" and self.original_overlay_image:
            delta = 0
            if platform.system() == "Linux":
                if event.num == 4: delta = 1
                elif event.num == 5: delta = -1
            else: delta = event.delta
            if delta > 0: self.current_scale *= 1.1
            elif delta < 0: self.current_scale *= 0.9
            self.current_scale = max(0.1, min(self.current_scale, 3.0))
            self.lbl_status.config(text=f"Status: Scalare {(self.current_scale*100):.0f}%")
            self.apply_transformations()

    def open_lasso_tool(self):
        if not self.display_image:
            messagebox.showwarning("Atentie", "Incarca intai imaginea de baza!")
            return
        lasso_app = LassoSelectorApp(self.root)
        self.root.wait_window(lasso_app.top)
        if lasso_app.result_image:
            self.overlay_image = lasso_app.result_image
            self.place_overlay_on_canvas()
        else:
            self.lbl_status.config(text="Status: Selecție anulată.")

    def place_overlay_on_canvas(self):
        if self.overlay_image.width > 500 or self.overlay_image.height > 500:
             self.overlay_image.thumbnail((500, 500))
        self.original_overlay_image = self.overlay_image.copy()
        self.current_scale = 1.0
        self.rotate_slider.set(0)
        self.tk_overlay = ImageTk.PhotoImage(self.overlay_image)
        cx = self.canvas.winfo_width() // 2
        cy = self.canvas.winfo_height() // 2
        self.overlay_id = self.canvas.create_image(cx, cy, image=self.tk_overlay, tags="overlay", anchor="center")
        self.interaction_mode = "move_object"
        self.rotate_slider.config(state=tk.NORMAL)
        self.lbl_status.config(text="Status: Obiect activ. Scroll pt zoom, Slider pt rotire.")

    def load_base_image(self):
        fn = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.png")])
        if fn:
            self.base_image = Image.open(fn).convert("RGB")
            self.base_image.thumbnail((1200, 900))
            self.display_image = self.base_image.copy()
            self.update_canvas(self.display_image)
            self.update_size_label()
            self.lbl_status.config(text="Status: Imagine încărcată.")
            self.interaction_mode = None
            self.rotate_slider.set(0)
            self.rotate_slider.config(state=tk.DISABLED)

    def update_canvas(self, pil_img):
        self.display_image = pil_img
        self.tk_image = ImageTk.PhotoImage(pil_img)
        self.canvas.delete("all")
        cx, cy = self.canvas.winfo_width(), self.canvas.winfo_height()
        if cx<10: cx=800
        if cy<10: cy=600
        ix, iy = max(0, (cx-pil_img.width)//2), max(0, (cy-pil_img.height)//2)
        self.canvas.create_image(ix, iy, image=self.tk_image, anchor="nw", tags="background")
        self.bg_coords = (ix, iy)

    def mode_add_text(self):
        if not self.display_image: return
        t = simpledialog.askstring("Text", "Text de inserat:")
        if t: self.text_to_place = t; self.interaction_mode = "text"; self.lbl_status.config(text="Status: Click pe imagine pt text.")

    def mode_local_compression(self):
        if not self.display_image: return
        self.interaction_mode = "select_zone"
        self.lbl_status.config(text="Status: Trage cu mouse-ul un DREPTUNGHI pentru a comprima zona interioară.")

    def on_canvas_click(self, e):
        if self.interaction_mode == "text":
            rx, ry = int(e.x - self.bg_coords[0]), int(e.y - self.bg_coords[1])
            if 0 <= rx < self.display_image.width and 0 <= ry < self.display_image.height:
                draw = ImageDraw.Draw(self.display_image)
                try: font = ImageFont.truetype("arial.ttf", 50)
                except: font = ImageFont.load_default()
                draw.text((rx, ry), self.text_to_place, fill="red", font=font)
                self.update_canvas(self.display_image)
                self.interaction_mode = None
                self.update_size_label()
        elif self.interaction_mode == "move_object":
            item = self.canvas.find_closest(e.x, e.y)
            if "overlay" in self.canvas.gettags(item):
                self.drag_data["item"] = item; self.drag_data["x"] = e.x; self.drag_data["y"] = e.y
        elif self.interaction_mode == "select_zone":
            self.zone_start = (e.x, e.y)
            if self.zone_rect_id: self.canvas.delete(self.zone_rect_id)
            self.zone_rect_id = self.canvas.create_rectangle(e.x, e.y, e.x, e.y, outline="red", width=2, dash=(4, 4))

    def on_canvas_drag(self, e):
        if self.interaction_mode == "move_object" and "item" in self.drag_data:
            dx, dy = e.x - self.drag_data["x"], e.y - self.drag_data["y"]
            self.canvas.move(self.drag_data["item"], dx, dy)
            self.drag_data["x"], self.drag_data["y"] = e.x, e.y
        elif self.interaction_mode == "select_zone" and self.zone_start:
            x1, y1 = self.zone_start
            self.canvas.coords(self.zone_rect_id, x1, y1, e.x, e.y)

    def on_canvas_release(self, e):
        if self.interaction_mode == "select_zone" and self.zone_start:
            x1, y1 = self.zone_start
            x2, y2 = e.x, e.y
            if self.zone_rect_id: self.canvas.delete(self.zone_rect_id)
            self.zone_rect_id = None; self.zone_start = None
            
            left_canvas, top_canvas = min(x1, x2), min(y1, y2)
            right_canvas, bottom_canvas = max(x1, x2), max(y1, y2)
            if (right_canvas - left_canvas) < 5 or (bottom_canvas - top_canvas) < 5: return

            img_left = max(0, int(left_canvas - self.bg_coords[0]))
            img_top = max(0, int(top_canvas - self.bg_coords[1]))
            img_right = min(self.display_image.width, int(right_canvas - self.bg_coords[0]))
            img_bottom = min(self.display_image.height, int(bottom_canvas - self.bg_coords[1]))
            
            self.apply_compression_local(img_left, img_top, img_right, img_bottom)
            self.interaction_mode = None 
            self.lbl_status.config(text="Status: Compresie locală aplicată.")

    def on_right_click(self, e):
        if self.interaction_mode == "move_object" and self.overlay_image:
            ox, oy = self.canvas.coords(self.overlay_id)
            top_left_x = ox - (self.overlay_image.width / 2)
            top_left_y = oy - (self.overlay_image.height / 2)
            paste_x = int(top_left_x - self.bg_coords[0])
            paste_y = int(top_left_y - self.bg_coords[1])
            
            if self.overlay_image.mode == 'RGBA':
                self.display_image.paste(self.overlay_image, (paste_x, paste_y), self.overlay_image)
            else:
                self.display_image.paste(self.overlay_image, (paste_x, paste_y))

            self.canvas.delete(self.overlay_id)
            self.overlay_id = None; self.overlay_image = None; self.original_overlay_image = None
            self.interaction_mode = None
            self.rotate_slider.config(state=tk.DISABLED)
            self.update_canvas(self.display_image)
            self.update_size_label()
            self.lbl_status.config(text="Status: Obiect fixat.")

    def apply_compression_global(self):
        if not self.display_image: return
        q = self.compress_slider.get()
        buf = io.BytesIO()
        self.display_image.save(buf, "JPEG", quality=q)
        s = len(buf.getvalue())/1024.0
        self.lbl_size_comp.config(text=f"Mărime fișier: {s:.1f} KB")
        buf.seek(0)
        self.display_image = Image.open(buf).convert("RGB")
        self.update_canvas(self.display_image)
        messagebox.showinfo("Compresie", f"Comprimat Global la Q={q}. Mărime: {s:.1f} KB")

    def apply_successive_compression(self):
        if not self.display_image: return
        iterations = self.successive_slider.get()
        quality = self.compress_slider.get()
        
        self.lbl_status.config(text=f"⏳ Se comprimă succesiv de {iterations} ori...", fg=COLORS["warning"])
        self.root.update()
        
        current_img = self.display_image
        for i in range(iterations):
            buffer = io.BytesIO()
            current_img.save(buffer, format="JPEG", quality=quality)
            buffer.seek(0)
            current_img = Image.open(buffer).convert("RGB")
        
        self.display_image = current_img
        self.update_canvas(self.display_image)
        self.update_size_label()
        
        messagebox.showinfo("Succes", f"Imaginea a fost re-salvată de {iterations} ori la calitatea {quality}%.")
        self.lbl_status.config(text=f"Status: Compresie Succesivă ({iterations}x) finalizată.", fg=COLORS["text_dim"])

    def apply_compression_local(self, x1, y1, x2, y2):
        try:
            crop = self.display_image.crop((x1, y1, x2, y2))
            quality = self.compress_slider.get()
            buffer = io.BytesIO()
            crop.save(buffer, format="JPEG", quality=quality)
            buffer.seek(0)
            degraded_crop = Image.open(buffer).convert("RGB")
            self.display_image.paste(degraded_crop, (x1, y1))
            self.update_canvas(self.display_image)
            self.update_size_label()
        except Exception as e:
            messagebox.showerror("Eroare", f"Nu s-a putut comprima zona: {e}")

    def run_ela(self):
        if not self.display_image: return
        
        orig = self.display_image.convert("RGB")
        ela_q = self.ela_quality.get()
        user_scale = self.ela_scale.get()
        
        buf = io.BytesIO()
        orig.save(buf, "JPEG", quality=ela_q)
        buf.seek(0)
        recomp = Image.open(buf).convert("RGB")
        
        ela_img = ImageChops.difference(orig, recomp)
        extrema = ela_img.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0: max_diff = 1 
        scale_factor = (255.0 / max_diff) * (user_scale / 20.0)
        
        # Amplificare pixel-level (mai precis)
        r, g, b = ela_img.split()
        scale_func = lambda i: i * scale_factor
        r = r.point(scale_func)
        g = g.point(scale_func)
        b = b.point(scale_func)
        ela_img = Image.merge("RGB", (r, g, b))
        ela_img = ImageEnhance.Color(ela_img).enhance(3.0) 
        
        top = Toplevel(self.root)
        top.title(f"Rezultat ELA - Mod Comparativ")
        top.geometry("1000x850")
        top.configure(bg="#000") 
        
        tk_ela = ImageTk.PhotoImage(ela_img)
        tk_orig = ImageTk.PhotoImage(orig)
        
        lbl_img = tk.Label(top, image=tk_ela, bg="#000", bd=0, cursor="hand2")
        lbl_img.pack(expand=True)
        lbl_img.img_ela = tk_ela
        lbl_img.img_orig = tk_orig
        
        def show_original(event): lbl_img.configure(image=lbl_img.img_orig)
        def show_ela(event): lbl_img.configure(image=lbl_img.img_ela)
            
        lbl_img.bind("<ButtonPress-1>", show_original)
        lbl_img.bind("<ButtonRelease-1>", show_ela)
        
        info_text = (f"Eroare Max: {max_diff} | Amplificare: {scale_factor:.1f}x\n"
                     f"COMPARARE: Ține CLICK STÂNGA apăsat pe imagine pentru a vedea ORIGINALUL.")
        tk.Label(top, text=info_text, bg="#000", fg="#aaa", font=("Segoe UI", 10)).pack(pady=10)

    def reset_to_original(self):
        if self.base_image:
            self.display_image = self.base_image.copy()
            self.update_canvas(self.display_image)
            self.canvas.delete("overlay")
            self.rotate_slider.config(state=tk.DISABLED)
            self.update_size_label()

    def save_image(self):
        if self.display_image:
            f = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg")])
            if f: self.display_image.save(f, quality=95)

    def open_fotoforensics(self):
        webbrowser.open("http://fotoforensics.com/")
        
    def open_forensically(self):
        webbrowser.open("https://29a.ch/photo-forensics/#error-level-analysis")

if __name__ == "__main__":
    root = tk.Tk()
    app = ForensicELAApp(root)
    root.mainloop()